# Generate Syntatic Dataset: PDF

In [1]:
import matplotlib.pyplot as plt
from fpdf import FPDF
from PIL import Image
import numpy as np
import pandas as pd

from faker import Faker
from sqlalchemy import create_engine

## Generate Syntatic Data

In [2]:
fake = Faker()

# Create synthetic data for a transaction log
data = []
for i in range(1, 1001):  
    data.append([
        i,  # transaction_id as unique integer
        fake.name(),  # username
        fake.email(),  # email
        fake.date_this_year(),  # transaction_date
        fake.credit_card_provider(),  # payment_method
        round(fake.random_number(), 2),  # amount_spent
        fake.random_element(elements=("Completed", "Pending", "Failed")),  # transaction_status
        fake.address(),  # delivery_address
        fake.random_element(elements=("Electronics", "Clothing", "Groceries")),  # product_category
        fake.random_number(digits=2),  # discount_applied
    ])

# Convert data to DataFrame
df = pd.DataFrame(data, columns=[
    'transaction_id', 'username', 'email', 'transaction_date', 'payment_method',
    'amount_spent', 'transaction_status', 'delivery_address', 'product_category', 'discount_applied'
])

# Save to CSV
df.to_csv("synthetic_data.csv", index=False)


In [ ]:
%pip install psycopg2-binary sqlalchemy faker

In [4]:
synthetic_data_df = pd.read_csv('synthetic_data.csv')

synthetic_data_df.sample(5)

,transaction_id,username,email,transaction_date,payment_method,amount_spent,transaction_status,delivery_address,product_category,discount_applied
211,212,Katherine Steele MD,matthewwright@example.net,2025-02-05,Maestro,33910165,Pending,"6528 Hayden River\nEast Angela, FM 81535",Groceries,63
345,346,Jessica Ritter,creynolds@example.com,2025-01-03,JCB 16 digit,828431841,Pending,Unit 8203 Box 0926\nDPO AE 97210,Electronics,3
969,970,Samantha Conley,qparker@example.org,2025-01-11,Mastercard,87594,Pending,"51135 Hill Field\nLewisview, AK 16379",Clothing,30
783,784,Jason Murray,thompsonpamela@example.net,2025-01-20,Mastercard,6,Failed,"1254 Martha Via\nTammyhaven, OH 54612",Electronics,12
20,21,Robert Butler,michael48@example.org,2025-01-01,JCB 16 digit,54,Failed,"0068 Terry Corner\nWest Melaniestad, ID 30168",Electronics,5


In [5]:
synthetic_data_df.columns

Index(['transaction_id', 'username', 'email', 'transaction_date',
       'payment_method', 'amount_spent', 'transaction_status',
       'delivery_address', 'product_category', 'discount_applied'],
      dtype='object')

In [6]:
# Convert the DataFrame to SQL insert statements and save to a file
with open('synthetic_data.sql', 'w') as f:
    # Write SQL commands for table creation
    f.write("""
    CREATE TABLE transactions (
        transaction_id INTEGER PRIMARY KEY,
        username VARCHAR(100),
        email VARCHAR(100),
        transaction_date TIMESTAMP,
        payment_method VARCHAR(50),
        amount_spent DECIMAL(10, 2),
        transaction_status VARCHAR(50),
        delivery_address TEXT,
        product_category VARCHAR(50),
        discount_applied BOOLEAN
    );
    """)

    # Write INSERT commands for each row in the DataFrame
    for index, row in df.iterrows():
        # Convert discount_applied to a boolean (True if > 50, False otherwise)
        discount_applied = 'TRUE' if row['discount_applied'] > 50 else 'FALSE'
        
        insert_statement = f"""
        INSERT INTO transactions (transaction_id, username, email, transaction_date, payment_method, amount_spent, transaction_status, delivery_address, product_category, discount_applied)
        VALUES ({row['transaction_id']}, '{row['username']}', '{row['email']}', '{row['transaction_date']}', '{row['payment_method']}', {row['amount_spent']}, '{row['transaction_status']}', '{row['delivery_address']}', '{row['product_category']}', {discount_applied});
        """
        f.write(insert_statement + '\n')

print("SQL file generated successfully!")


SQL file generated successfully!
